In [2]:
### Set 1

import pandas as pd

# df = pd.read_csv('DS_Sample_1.csv', sep=',')
df = pd.read_csv('../../../datasets/level2/DS_Sample_1.csv', sep=',')
df['CUST_ID']=df['CUST_ID'].astype('int')

df['CREDIT_LIMIT']=df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].mean())
df['MINIMUM_PAYMENTS']=df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].mean())

In [3]:
# 1번 문제

mylist=list(set(df.TENURE.tolist()))

corrs=[]
for mylis in mylist:
    df2=df[df['TENURE']==mylis]
    corrs.append(df2[['BALANCE', 'CREDIT_LIMIT']].corr().iloc[1,0])

df3=pd.DataFrame({'TENURE':mylist,'corrs':corrs})

answer=df3.sort_values(by='corrs',ascending=False).corrs.iloc[0]
answer

0.9484046188238867

In [14]:
# 2번 문제

df4=df.copy()
df4=df4.drop(['CUST_ID'],axis=1)
for col in df4.columns.tolist():
    df4[col]= (df[col] - df[col].mean()) / df[col].std()

print(df4.head(2))

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

for n_cluster in [2,3,4,5]:
    cluster = KMeans(n_clusters=n_cluster, random_state=1234)
    cluster_labels = cluster.fit_predict(df4)
    silhouette_avg = silhouette_score(df4, cluster_labels)
    print(n_cluster, ':',silhouette_avg)

    BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0 -0.848343          -0.419669  -0.441715         -0.373861   
1  0.282650           0.012125  -0.468782         -0.373861   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0               -0.395103     -0.482113            -0.872264   
1               -0.470068      1.877528            -1.281917   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                   -0.803919                         -0.719260   
1                   -0.803919                         -0.923941   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0               -0.684358         -0.457689      -0.563834     -1.161088   
1                0.513236          0.065384      -0.627743      0.149950   

   PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT    TENURE  
0 -0.557117         -0.443503         -0.465311  0.282288  
1  0.360394         -0.086116          0.331426  0.282288  
2 : 0.

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df4)
pd.DataFrame(scaler.transform(df4), columns=df4.columns).head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


In [18]:
# 위에 따라 clsuter = 2를 선택

cluster = KMeans(n_clusters=2, random_state=1234)
df['label'] = cluster.fit_predict(df4)
answer=df.groupby(by='label').mean()['ONEOFF_PURCHASES'].max()
answer

3946.187525252525

In [19]:
# 3번 문제

from sklearn import tree
df=df.drop(['label'],axis=1)

df_train=df[df['CUST_ID']%4!=0]
df_test=df[df['CUST_ID']%4==0]

print(df_train.size, df_test.size)

clf = tree.DecisionTreeRegressor(max_depth=5, random_state=1234)
dlf = clf.fit(df_train.drop(['CUST_ID', 'ONEOFF_PURCHASES'],axis=1),df_train['ONEOFF_PURCHASES'])
df_test['predict']=clf.predict(df_test.drop(['CUST_ID', 'ONEOFF_PURCHASES'], axis=1))

answer = ( ( (df_test['ONEOFF_PURCHASES']-df_test['predict']) ** 2).mean() ) ** (1/2)
answer
answer = ( ( (df_test['ONEOFF_PURCHASES']-df_test['predict']) ** 2).mean() ) ** (1/2)
answer

13536 4464


C:\Users\Lee\AppData\Local\Temp\ipykernel_8992\2469323490.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predict']=clf.predict(df_test.drop(['CUST_ID', 'ONEOFF_PURCHASES'], axis=1))


2316.0072386657453